# Working with Time Zones

## Python's Time Zone Model

The `tzinfo` class is an abstract base class, and you are required to implement three methods:

```python
class tzinfo:
    def tzname(self, dt):
        raise NotImplementedError()

    def utcoffset(self, dt):
        raise NotImplementedError()
        
    def dst(self, dt):
        raise NotImplementedError()
        
    ...
```

- `tzname`: Return the name or abbreviation for the time zone at the given datetime.
- `utcoffset`: Return a `datetime.timedelta` representing the offset from UTC at the given datetime.
- `dst`: Return a `datetime.timedelta` representing the amount of the daylight saving time offset at a given time. *This method is rarely useful*.

The reason these are *methods* rather than fixed values is that a `tzinfo` represents a mapping between a datetime and the set of rules that applies at that time in this zone. The same concrete `tzinfo` class is used for many `datetime`s.

### Example: Eastern Time

In [1]:
from dateutil import tz
from datetime import datetime, timedelta

In [2]:
def print_tzinfo(dt):
    """Convenience function for printing the time zone information of a datetime"""
    print("dt.tzname: " + str(dt.tzname()))
    print("dt.utcoffset: " + str(dt.utcoffset() / timedelta(hours=1)) + " hours")
    print("dt.dst: " + str(dt.dst() / timedelta(hours=1)) + " hours")

In [3]:
EASTERN = tz.gettz("America/New_York")

In [4]:
dt_std = datetime(2019, 3, 1, 12, tzinfo=EASTERN)

In [5]:
print_tzinfo(dt_std)

dt.tzname: EST
dt.utcoffset: -5.0 hours
dt.dst: 0.0 hours


In [6]:
dt_dst = datetime(2019, 4, 1, 12, tzinfo=EASTERN)

In [7]:
print_tzinfo(dt_dst)

dt.tzname: EDT
dt.utcoffset: -4.0 hours
dt.dst: 1.0 hours


### Exercise: Implement a UTC time zone class

The simplest `tzinfo` classes to implement are fixed offsets from UTC and UTC itself, which have offsets that do not change as a function of `datetime`. To practice a basic tzinfo implementation, implement a class `UTC` representing the UTC time zone.

In [8]:
from datetime import tzinfo

In [9]:
from datetime import timezone

In [10]:
datetime(2019, 1, 1, tzinfo=timezone.utc).dst()

In [11]:
class UTC(tzinfo):
    def tzname(self, dt):
        return "UTC"
    
    def utcoffset(self, dt):
        return timedelta(0)
    
    def dst(self, dt):
        return None

In [12]:
# Tests
test_cases = [
    datetime(1, 1, 1),
    datetime(1857, 2, 7),
    datetime(1970, 1, 1),
    datetime(2010, 3, 21, 2, 16)
]

utc = UTC()
for dt in test_cases:
    dt_act = dt.replace(tzinfo=utc)
    dt_exp = dt.replace(tzinfo=timezone.utc)
    
    assert dt_act.tzname() == dt_exp.tzname(), f'tznames must match for {dt}'
    assert dt_act.utcoffset() == dt_exp.utcoffset(), f'utcoffset must match for {dt}'
    assert dt_act.dst() == dt_exp.dst(), f'dst must match for {dt}'

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [13]:
from helper_functions import print_dt_tzinfo

In [14]:
dt = datetime(2017, 8, 11, 14, tzinfo=tz.gettz('America/New_York'))
print_dt_tzinfo(dt)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you have a naive wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [15]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

2017-08-11 14:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [16]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


### `pytz`

In `pytz`, `datetime.astimezone()` still works as expected:

In [17]:
import pytz

In [18]:
print_dt_tzinfo(dt.astimezone(pytz.timezone('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


But the constructor and `.replace` methods fail horribly:

In [19]:
print_dt_tzinfo(dt.replace(tzinfo=pytz.timezone('America/Los_Angeles')))

2017-08-11 14:00:00-0753
    tzname:   LMT;      UTC Offset:  -7.88h;        DST:      0.0h


This is because `pytz` uses a different time zone model. `pytz`'s time zone model implements the `tzinfo` interface *statically*, which is to say that the `tzname`, `utcoffset` and `dst` are all calculated eagerly, when the `tzinfo` is attached to the `datetime`.

In order to accomplish this, `pytz` expects all `tzinfo` objects to be attached to the `datetime` *by the time zone object itself*, using the `localize()` method:

In [20]:
EASTERN_p = pytz.timezone('America/New_York')
dt_p = EASTERN_p.localize(datetime(2017, 8, 11, 14))
print_dt_tzinfo(dt_p)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


This also means that unlike with normal `tzinfo` objects, after you've done some arithmetic on a `pytz`-aware `datetime` object, you must `normalize` it:

In [21]:
print('dateutil:')
print_dt_tzinfo(dt + timedelta(days=180))
print('')
print('pytz')
print_dt_tzinfo(dt_p + timedelta(days=180))

dateutil:
2018-02-07 14:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

pytz
2018-02-07 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


In [22]:
print_dt_tzinfo(EASTERN_p.normalize(dt_p + timedelta(days=180)))

2018-02-07 13:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


### `datetime.now` and `datetime.fromtimestamp`

The alternate constructors `now()` and `fromtimestamp` both take an optional argument `tz`:


```python
    def now(tz=None):
        """Get the datetime representing the current time"""
        # ...

    def fromtimestamp(self, timestamp, tz=None):
        """ Return the datetime corresponding to a POSIX timestamp """
        # ...
```

If the `tz` parameter is not passed, they will return a *naïve* datetime, representing the time in your system local time.

In [23]:
datetime.fromtimestamp(1577854800.0)

datetime.datetime(2020, 1, 1, 0, 0)

If you want an *aware* timezone from a timestamp (or the current time), pass it to the `tz` parameter, and it will calculate the correct `datetime` matching that time zone. This works with both `pytz` and `dateutil.tz`:

In [24]:
print("pytz:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN_p))
print("")
print("dateutil:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN))

pytz:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

dateutil:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: There are also the semi-deprecated `datetime.utcnow()` and `datetime.utcfromtimestamp()` functions. These  return a *naïve* datetime expressed in UTC. It is almost always better to simply pass `tz=UTC` (where `UTC` is some time zone object), which will give you an *aware* `datetime`.

### Exercise: Current time in multiple time zones


To practice using time zones, try writing a function that takes as inputs a list of IANA time zones and prints the time in each time zone.

So, for example:

```python
>>> now_in_zones(['Asia/Tokyo',
...               'Europe/Berlin',
...               'America/New_York',
...               'America/Los_Angeles'])
```
```
Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
```

In [25]:
def now_in_zones(tz_list):
    # For now I'll fib and pretend the current datetime is 2020
    dt_utc = datetime(2020, 1, 1, tzinfo=tz.gettz('America/New_York'))
    for tzstr in tz_list:
        dt = dt_utc.astimezone(tz.gettz(tzstr))
        print(f"{tzstr + ':':<25} {dt}")
        
now_in_zones(['Asia/Tokyo',
              'Europe/Berlin',
              'America/New_York',
              'America/Los_Angeles'])
    

Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
